In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [2]:
program_id = PublicKey("5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [3]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [4]:
bump = 4;
avatar_mint = PublicKey("2prH8eozjsUyxo1KJf3RiDv5b9TLiuGTuqBuNLys6DTQ")
creator_pda, _ = PublicKey.find_program_address([b"avatar_mint_authority_pda"], AVATAR_PROGRAM_ID)
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
collection_pda, _ = PublicKey.find_program_address([b"Dsol Dao_seed", bytes(creator_pda)], program_id)
payer_collection_pda, _ = PublicKey.find_program_address([b"payer_collection_dsol", bytes(payer_keypair.public_key), bytes(creator_pda)], program_id)
container_pda, container_pda_bump = PublicKey.find_program_address( [b"Dsol Dao_seed", bump.to_bytes(4, "big"), bytes(creator_pda), bytes(payer_keypair.public_key)], program_id);
rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
metadata_pda, _metadata_nonce = PublicKey.find_program_address([b"metadata", bytes(METAPLEX_PROGRAM_ID), bytes(avatar_mint)], METAPLEX_PROGRAM_ID);

accountSchema = CStruct(
    "collection_address" / U8[32],
    "mint_address" / U8[32],
    "price" / U64,
    "owner" / U8[32],
    "is_used" / Bool
)

container_data = getAccountInfo(client, container_pda)

container_data

Container(collection_address=ListContainer([171, 248, 234, 40, 134, 242, 121, 159, 158, 33, 203, 242, 155, 157, 123, 130, 223, 124, 13, 153, 35, 199, 169, 178, 255, 209, 29, 72, 169, 203, 205, 124]), mint_address=ListContainer([27, 29, 180, 52, 20, 197, 231, 187, 242, 234, 198, 199, 115, 106, 176, 63, 110, 193, 247, 234, 170, 83, 188, 18, 24, 221, 35, 190, 159, 194, 185, 211]), price=11028225974486433791, owner=ListContainer([230, 93, 161, 21, 203, 134, 227, 249, 98, 231, 6, 246, 65, 15, 155, 136, 121, 241, 203, 109, 115, 234, 247, 41, 223, 105, 164, 97, 1, 0, 0, 0]), is_used=False)

In [5]:
owner_account_meta = AccountMeta(PublicKey(container_data.owner), False, True)
owner_account_meta

AccountMeta(pubkey=GWFaSt13B1kFSpKLnEvyM9oohZR1yqi86vzx6Ut5H2EB, is_signer=False, is_writable=True)

In [6]:
accounts = [
    AccountMeta(payer_keypair.public_key, True, True),
    AccountMeta(avatar_mint, False, False),
    AccountMeta(container_pda, False, True),
    AccountMeta(collection_pda, False, True),
    AccountMeta(creator_pda, False, False),
    AccountMeta(instructions.get_associated_token_address(payer_keypair.public_key, avatar_mint), False, True),
    AccountMeta(instructions.get_associated_token_address(container_pda, avatar_mint), False, True),
    spl_program_meta,
    AccountMeta(payer_collection_pda, False, True),



    spl_program_meta,
]

In [7]:
transaction = Transaction()
transaction.add(
    TransactionInstruction(
        accounts,
        program_id,
        bytes([3,0,0,bump])
    )
)

f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: insufficient account keys for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'NotEnoughAccountKeys']}, 'logs': ['Program 5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE invoke [1]', 'Program 5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE consumed 2939 of 200000 compute units', 'Program 5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE failed: insufficient account keys for instruction'], 'unitsConsumed': 0}}